# TP Elasticsearch


**Alumno: Ivan Fabio Gutierrez Pinedo DNI 33437838**

Utilizando el archivo G20 provisto durante la clase (https://bit.ly/g20_Diplo2020), indexar los tweets del archivo, asegurándose de definir el mapping de forma correcta para los campos de interés. Notar que el mapping correcto no sólo depende del tipo de datos, sino de las consultas que deben hacerse sobre los mismos. De ser necesario o conveniente, se podría elegir indexar un campo de más de una forma.
Los campos de interés son:
*  “text”, “created_at”, “id”, “user.location”, “user.followers_count”, “place.bounding_box”,  “source”, “entities.hashtags”, “timestamp_ms”, “retweeted”
Las consultas de interés son:
1. Poder buscar teniendo en cuenta el patrón de mayúsculas o minúsculas, por ejemplo: “President” o “PRESIDENT” en el campo ‘text’, solo debe devolver documentos que tengan la palabra escrita con el mismo patrón de mayúsculas y minúsculas.
2. Poder buscar por palabras que compartan una misma raíz, por ejemplo: “pray”, haría un match con tweets que contengan: “prays”, “prayer”, “praying”, “prayers” en el campo ‘text’.
3. Consulta para poder buscar una ubicación en el campo ‘user.location’ y que el matching sea exacto, por ejemplo para las consultas “York” y “California”, no debería devolver documentos en donde user.location sea “New York” o “LA, California” por ejemplo.
4. Consulta para poder buscar tweets cuyos usuarios hayan abierto sus cuentas en un rango de fechas.
5. Consulta para poder buscar tweets que hayan sido posteados en un área específica,por ejemplo, en el área de “Washington DC” usando el campo “place.bounding_box”.

Investigacion para realizar el TP 
* https://www.youtube.com/watch?v=ma3BC8aPBfE&ab_channel=AmineM.Boulouma

In [296]:
from elasticsearch import Elasticsearch, helpers
import os, uuid
import json
from datetime import datetime
import dateutil.parser

In [304]:
# Configuration

mapping = {
  "properties": {
    "text": {
      "type": "text",
      "analyzer": "my_analyzer",
      "fields": {
        "case_sensitive": {
          "type": "text",
          "analyzer": "case_sensitive"
        }        
      }      
    },
    "created_at": {
      "type": "date",
      "format": "E MMM dd HH:mm:ss Z yyyy"
    },
    "user": {
      "properties": {
        "location": { "type": "keyword" }
      }
    },
    "place": {
      "properties": {
        "bounding_box": {
          "type": "geo_shape"
          }
      }
    }  
  }
}

setting = {
  "settings": {
    "analysis": {
      "analyzer": {
        "my_analyzer": {
          "tokenizer": "standard",
          "filter": [
            "lowercase",
            "my_stemmer"
          ]
        },
        "stem_analysis": {
          "tokenizer": "whitespace",
          "filter": [ "stemmer" ]
        },
      "case_sensitive" : {
             "type" : "custom",
             "tokenizer":    "standard",
             "filter": ["stop", "porter_stem" ]                    
         }
      },
      "filter": {
        "my_stemmer": {
          "type": "stemmer",
          "language": "english"#"light_english"#"english"
        }
      }
    },
    "index.mapping.total_fields.limit": 2000,
  }
}

In [305]:
mapping

{'properties': {'text': {'type': 'text',
   'analyzer': 'my_analyzer',
   'fields': {'case_sensitive': {'type': 'text',
     'analyzer': 'case_sensitive'}}},
  'created_at': {'type': 'date', 'format': 'E MMM dd HH:mm:ss Z yyyy'},
  'user': {'properties': {'location': {'type': 'keyword'}}},
  'place': {'properties': {'bounding_box': {'type': 'geo_shape'}}}}}

In [306]:
index_name = 'g20_index'
es = Elasticsearch(HOST="http://localhost", PORT=9200)
exist_index = es.indices.exists(index_name)
if exist_index :
    es.indices.delete(index_name)
es.indices.create(index_name, body=setting)



{'acknowledged': True, 'shards_acknowledged': True, 'index': 'g20test'}

In [307]:
es.indices.put_mapping(body=mapping, index=index_name)

{'acknowledged': True}

In [308]:
es.indices.put_settings(json.dumps(index_setting), index=index_name)
es.indices.get_settings(index_name)

{'g20test': {'settings': {'index': {'routing': {'allocation': {'include': {'_tier_preference': 'data_content'}}},
    'mapping': {'total_fields': {'limit': '2000'}},
    'number_of_shards': '1',
    'provided_name': 'g20test',
    'creation_date': '1611447431632',
    'analysis': {'filter': {'my_stemmer': {'type': 'stemmer',
       'language': 'english'}},
     'analyzer': {'my_analyzer': {'filter': ['lowercase', 'my_stemmer'],
       'tokenizer': 'standard'},
      'stem_analysis': {'filter': ['stemmer'], 'tokenizer': 'whitespace'},
      'case_sensitive': {'filter': ['stop', 'porter_stem'],
       'type': 'custom',
       'tokenizer': 'standard'}}},
    'number_of_replicas': '1',
    'uuid': 'AmgD7lv4QDW3DxhvC0efHg',
    'version': {'created': '7100099'}}}}}

In [309]:
def get_data_from_file(file_name):
    if "/" in file_name or chr(92) in file_name:
        file = open(file_name, encoding="utf8", errors='ignore')
    else:
        # use the script_path() function to get path if none is passed
        file = open('./' + str(file_name), encoding="utf8", errors='ignore')
    data = [line.strip() for line in file]
    file.close()
    return data

def bulk_g20_ndjson_data(json_file, _index):
    json_list = get_data_from_file(json_file)
    index = None
    dict_index = None
    for doc in json_list:
        if '{"index"' in doc:
            dict_index = json.loads(doc)
        if '{"index"' not in doc and index != None:        
            yield {
                "_index": _index,
                #"_type": doc_type,
                "_id": dict_index.get('index').get('_id'),
                "_source": doc
            }
            dict_index = None
            
def bulk_g20_from_dict_data(g20_dict, _index):      
    for key, doc in g20_dict.items():
          
        yield {
            "_index": _index,
            #"_type": doc_type,
            "_id": key,
            "_source": doc
        }
        
def convert_g20_to_dict(file_name):    
    data = get_data_from_file('g20.json')
    index_list = []
    doc_list = []
    for row in data:
        if '{"index"' in row:
            dict_index = json.loads(row)
            index_list.append(dict_index)
        else :
            doc_list.append(row)


    g20_dict = { index_list[i].get('index').get('_id') : doc_list[i] for i in range(0, len(index_list)) } 
    return g20_dict


In [310]:
data = convert_g20_to_dict('g20.json')
helpers.bulk(es, bulk_g20_from_dict_data(data, index_name))

BulkIndexError: ('7 document(s) failed to index.', [{'index': {'_index': 'g20test', '_type': '_doc', '_id': '1059862707782987777', 'status': 400, 'error': {'type': 'mapper_parsing_exception', 'reason': 'failed to parse field [place.bounding_box] of type [geo_shape]', 'caused_by': {'type': 'x_content_parse_exception', 'reason': 'Failed to build [geojson] after last required field arrived', 'caused_by': {'type': 'illegal_argument_exception', 'reason': 'first and last points of the linear ring must be the same (it must close itself): x[0]=72.74484 x[3]=73.003648 y[0]=18.845343 y[3]=18.845343'}}}, 'data': '{"quote_count": 0, "contributors": null, "truncated": false, "text": "@realDonaldTrump  : Mr. Trump please make America great again, Vote for Obama\\u2019s party.", "is_quote_status": false, "in_reply_to_status_id": null, "reply_count": 0, "id": 1059862707782987777, "favorite_count": 0, "source": "<a href=\\"http://twitter.com/download/iphone\\" rel=\\"nofollow\\">Twitter for iPhone</a>"

---
1. Poder buscar teniendo en cuenta el patrón de mayúsculas o minúsculas, por ejemplo: “President” o “PRESIDENT” en el campo ‘text’, solo debe devolver documentos que tengan la palabra escrita con el mismo patrón de mayúsculas y minúsculas.


In [273]:
word = "PRESIDENT"
search_body = {
  "query": {
    "match": { "text.case_sensitive": word } 
  } 
  , "_source": ["text"]

}
es.search(body=search_body, index=index_name)

{'took': 5,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 6, 'relation': 'eq'},
  'max_score': 8.57464,
  'hits': [{'_index': 'g20test',
    '_type': '_doc',
    '_id': '1059864699968372737',
    '_score': 8.57464,
    '_source': {'text': '@realDonaldTrump Great Job Mr. PRESIDENT! Finally.. https://t.co/QePjSzlIi8'}},
   {'_index': 'g20test',
    '_type': '_doc',
    '_id': '1059868987855724544',
    '_score': 7.782443,
    '_source': {'text': '@WhiteHouse @realDonaldTrump #HELLO..MY PRESIDENT PRAY FOR..EVER..CONQUER..SMILE..SECOND..45.'}},
   {'_index': 'g20test',
    '_type': '_doc',
    '_id': '1059869552916512771',
    '_score': 7.3309164,
    '_source': {'text': '@realDonaldTrump \nPOTUS:  It is time for obstructionists to work with you and accept that YOU are the PRESIDENT!\nLO… https://t.co/ETZ0A7d5y4'}},
   {'_index': 'g20test',
    '_type': '_doc',
    '_id': '1059869654993256449',
    '_score': 6.928910

In [274]:
word = "President"
search_body = {
  "query": {
    "match": { "text.case_sensitive": word } 
  } 
  , "_source": ["text"]

}
es.search(body=search_body, index=index_name)

{'took': 4,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 580, 'relation': 'eq'},
  'max_score': 3.9671803,
  'hits': [{'_index': 'g20test',
    '_type': '_doc',
    '_id': '1059868781495894021',
    '_score': 3.9671803,
    '_source': {'text': '@realDonaldTrump Come on, Mr President!!!!'}},
   {'_index': 'g20test',
    '_type': '_doc',
    '_id': '1059868897493557248',
    '_score': 3.9671803,
    '_source': {'text': '@realDonaldTrump Weak Baby President.'}},
   {'_index': 'g20test',
    '_type': '_doc',
    '_id': '1059869201450569734',
    '_score': 3.9671803,
    '_source': {'text': '@realDonaldTrump Ok Mr. President!'}},
   {'_index': 'g20test',
    '_type': '_doc',
    '_id': '1059868532975026178',
    '_score': 3.8114316,
    '_source': {'text': '@realDonaldTrump Thank you President Klump!'}},
   {'_index': 'g20test',
    '_type': '_doc',
    '_id': '1059868985058041856',
    '_score': 3.8114316,
    '_sou

---
2. Poder buscar por palabras que compartan una misma raíz, por ejemplo: “pray”, haría un match con tweets que contengan: “prays”, “prayer”, “praying”, “prayers” en el campo ‘text’.


In [251]:
#Testing custom analyzer
analisis = {
  "analyzer" : "my_analyzer",
  "text" : "pray prayed praying prayer prayers"
}
es.indices.analyze(body=analisis, index=index_name)

{'tokens': [{'token': 'prai',
   'start_offset': 0,
   'end_offset': 4,
   'type': '<ALPHANUM>',
   'position': 0},
  {'token': 'prai',
   'start_offset': 5,
   'end_offset': 11,
   'type': '<ALPHANUM>',
   'position': 1},
  {'token': 'prai',
   'start_offset': 12,
   'end_offset': 19,
   'type': '<ALPHANUM>',
   'position': 2},
  {'token': 'prayer',
   'start_offset': 20,
   'end_offset': 26,
   'type': '<ALPHANUM>',
   'position': 3},
  {'token': 'prayer',
   'start_offset': 27,
   'end_offset': 34,
   'type': '<ALPHANUM>',
   'position': 4}]}

In [275]:
word = "praying"
search_body = {
  "query": {
    "match": { "text": word } 
  } 
  , "_source": ["text"]

}
es.search(body=search_body, index=index_name)

{'took': 5,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 11, 'relation': 'eq'},
  'max_score': 7.68461,
  'hits': [{'_index': 'g20test',
    '_type': '_doc',
    '_id': '1059868987855724544',
    '_score': 7.68461,
    '_source': {'text': '@WhiteHouse @realDonaldTrump #HELLO..MY PRESIDENT PRAY FOR..EVER..CONQUER..SMILE..SECOND..45.'}},
   {'_index': 'g20test',
    '_type': '_doc',
    '_id': '1059864553037684736',
    '_score': 7.4821825,
    '_source': {'text': '@WhiteHouse @realDonaldTrump The American People are praying for you Mr President 🙏🇺🇸'}},
   {'_index': 'g20test',
    '_type': '_doc',
    '_id': '1059862775965446144',
    '_score': 7.4586577,
    '_source': {'text': '@POTUS   🙏🏻🙏🏻Pray For President Trump🙏🏻🙏🏻🙏🏻\n🙏🏻🙏🏻🙏🏻🙏🏻Pray For America🙏🏻🙏🏻🙏🏻🙏🏻🙏🏻\nIn the Mighty Name of Jesus Release… https://t.co/HY2yIhB7LR'}},
   {'_index': 'g20test',
    '_type': '_doc',
    '_id': '1059867694256795655',
    '_score

---
3. Consulta para poder buscar una ubicación en el campo ‘user.location’ y que el matching sea exacto, por ejemplo para las consultas “York” y “California”, no debería devolver documentos en donde user.location sea “New York” o “LA, California” por ejemplo.


In [277]:
location_string = 'California'
search_body = {
  "query": {
    "match": {
      "user.location": location_string
    }
  }
  , "_source": ["id", "user.location"]
}

es.search(body=search_body, index=index_name)

{'took': 4,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 5, 'relation': 'eq'},
  'max_score': 6.9929314,
  'hits': [{'_index': 'g20test',
    '_type': '_doc',
    '_id': '1059867162557394944',
    '_score': 6.9929314,
    '_source': {'id': 1059867162557394944,
     'user': {'location': 'California'}}},
   {'_index': 'g20test',
    '_type': '_doc',
    '_id': '1059863755603947520',
    '_score': 6.9929314,
    '_source': {'id': 1059863755603947520,
     'user': {'location': 'California'}}},
   {'_index': 'g20test',
    '_type': '_doc',
    '_id': '1059867742466039808',
    '_score': 6.9929314,
    '_source': {'id': 1059867742466039808,
     'user': {'location': 'California'}}},
   {'_index': 'g20test',
    '_type': '_doc',
    '_id': '1059868585873395713',
    '_score': 6.9929314,
    '_source': {'id': 1059868585873395713,
     'user': {'location': 'California'}}},
   {'_index': 'g20test',
    '_type': '_doc',
  

---
4. Consulta para poder buscar tweets cuyos usuarios hayan abierto sus cuentas en un rango de fechas.


In [295]:
date_format ="%a %b %d %H:%M:%S %z %Y"

date_to = dateutil.parser.parse("2018-11-05T08:27:18-0000")
date_from = dateutil.parser.parse("2018-11-07T08:27:18-0000")

date_to_str = date_to.strftime(date_format)
date_from_str = date_from.strftime(date_format)

print(date_to_str)
print(date_from_str)

Mon Nov 05 08:27:18 +0000 2018
Wed Nov 07 08:27:18 +0000 2018


In [294]:
search_body = {
  "query": {
    "bool": {
      "must": [
        {
          "range": {
            "created_at": {
              "gte": date_to_str,
              "lte": date_from_str
            }
          }
        }
      ]
    }
  }
  , "_source": ["created_at"]
}

es.search(body=search_body, index=index_name)

{'took': 15,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 9567, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'g20test',
    '_type': '_doc',
    '_id': '1059866770843025408',
    '_score': 1.0,
    '_source': {'created_at': 'Tue Nov 06 17:55:04 +0000 2018'}},
   {'_index': 'g20test',
    '_type': '_doc',
    '_id': '1059866771212115971',
    '_score': 1.0,
    '_source': {'created_at': 'Tue Nov 06 17:55:04 +0000 2018'}},
   {'_index': 'g20test',
    '_type': '_doc',
    '_id': '1059866771006676992',
    '_score': 1.0,
    '_source': {'created_at': 'Tue Nov 06 17:55:04 +0000 2018'}},
   {'_index': 'g20test',
    '_type': '_doc',
    '_id': '1059866771547738113',
    '_score': 1.0,
    '_source': {'created_at': 'Tue Nov 06 17:55:04 +0000 2018'}},
   {'_index': 'g20test',
    '_type': '_doc',
    '_id': '1059866772696940544',
    '_score': 1.0,
    '_source': {'created_at': 'Tue Nov 06 17:55:04 +000

---
5. Consulta para poder buscar tweets que hayan sido posteados en un área específica,por ejemplo, en el área de “Washington DC” usando el campo “place.bounding_box”.